# A1: Implementação dos exercícios de modelagem de PLI

### Aluna: Sofia Lakschevitz
### Professor: Vincent Gerard Y. Guigues

## Importando a biblioteca Mosek

In [16]:
import mosek
import mosek.fusion as mf
import mosek.fusion.pythonic

In [ ]:
# Transportation Problem: List 5 Exercise 1
# Two factories (U1, U2) and three clients (E1, E2, E3)

with mf.Model("list5_exercise1") as M51:
    # Indexes
    factories = [0, 1]  # U1, U2
    clients = [0, 1, 2] # E1, E2, E3

    # Decision variables x[i,j] >= 0 and x is an integer
    x = M51.variable("x", [len(factories), len(clients)], mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Costs matrix
    costs = [[1.0, 1.5, 3.5],
             [2.0, 1.0, 2.0]]

    # Ojective function: minimize total cost
    M51.objective("MinimizeCost", mf.ObjectiveSense.Minimize, mf.Expr.dot(costs, x))

    # Constraint: demand for each client
    demands = [100.0, 200.0, 300.0]
    for j in clients:
        M51.constraint(f"Demand{j+1}", mf.Expr.add([x.index(i, j) for i in factories]), mf.Domain.equalsTo(demands[j]))

    # Constraint: capacity for each factory
    capacities = [400.0, 300.0]
    for i in factories:
        M51.constraint(f"Capacity{i+1}", mf.Expr.add([x.index(i, j) for j in clients]), mf.Domain.lessThan(capacities[i]))

    # Solving the model
    M51.solve()

    # Display solution
    print("Minimal cost:", M51.primalObjValue())

    print("\nOptimal quantities of product to be transported from factory i to client j:")
    for i in factories:
        for j in clients:
            print(f"x[{i+1},{j+1}] = {x.index(i,j).level()[0]}")

    print("\nExcedents in factories:")
    for i in factories:
        print(f"Excedent Factory {i+1} = {capacities[i] - (x.index(i, 0).level()[0] + x.index(i, 1).level()[0] + x.index(i, 2).level()[0])}")



Minimal cost: 1000.0

Optimal quantities of product to be transported from factory i to client j:
x[1,1] = 100.0
x[1,2] = 200.0
x[1,3] = 0.0
x[2,1] = 0.0
x[2,2] = 0.0
x[2,3] = 300.0

Excedents in factories:
Excedent U1 = 100.0
Excedent U2 = 0.0


In [ ]:
# Mixture Problem: List 5 Exercise 2

with mf.Model("list5_exercise2") as M52:
    # Indexes
    fertilizer = [0, 1, 2] # Ni, KCl, P
    products = [0, 1]  # Product1, Product2

    # Decision variables
    # p[i] = amount of product i produced, p[i] >= 0 and p is an integer
    p = M52.variable("p", [len(products)], mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # x[i,j] = amount of fertilizer used for the production of product i, x[i,j] >= 0 and x is an integer
    x = M52.variable("x", [len(fertilizer), len(products)], mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Costs matrix
    costs = [[1.0, 1.5, 3.5],
             [2.0, 1.0, 2.0]]

    # Ojective function: minimize total cost
    M51.objective("MinimizeCost", mf.ObjectiveSense.Minimize, mf.Expr.dot(costs, x))

    # Constraint: fertilizers stock
    fertilizer_stocks = [18.0, 19.0, 4.0]
    for i in fertilizer:
        M52.constraint(f"FertilizerStock{i+1}", mf.Expr.add([x.index(i, j) for j in products]), mf.Domain.lessThan(fertilizer_stocks[i]))

    # Constraint: fertilizer requirements for each product
    p1 = x11 + x21

    fertilizer_requirements = [[1.0, 2.0],
                                [3.0, 1.0],
                                [2.0, 1.0]]
    for j in factories:
        M51.constraint(f"Capacity_U{i+1}", mf.Expr.add([x.index(i, j) for j in clients]), mf.Domain.lessThan(capacities[i]))

    # Resolver
    M51.solve()

    # Exibir solução
    print("Minimal cost:", M51.primalObjValue())

    print("\nOptimal quantities of product to be transported from factory i to client j:")
    for i in factories:
        for j in clients:
            print(f"x[{i+1},{j+1}] = {x.index(i,j).level()[0]}")

    print("\nExcedents in factories:")
    for i in factories:
        print(f"Excedent U{i+1} = {capacities[i] - (x.index(i, 0).level()[0] + x.index(i, 1).level()[0] + x.index(i, 2).level()[0])}")



In [50]:
# Transportation Problem - List 5 Exercise 3

with mf.Model("list5_exercise3") as M53:
    # Indexes
    u_factories = [0, 1]  # U1, U2
    v_factories = [0, 1, 2] # V1, V2, V3

    # Decision variables x[i,j] >= 0 and x is an integer
    # x[i,j] = amount of engines transported from u_factory i to v_factory j
    x = M53.variable("x", [len(u_factories), len(v_factories)], mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Costs of transport matrix
    # costs[i][j] = cost of transporting from u_factory i to v_factory j
    costs = [[38, 27, 48],
             [37, 58, 45]]

    # Ojective function: minimize total cost
    M53.objective("MinimizeCost", mf.ObjectiveSense.Minimize, mf.Expr.dot(costs, x))

    # Constraint: demand for each v_factory j
    demands = [5, 4, 3]
    for j in v_factories:
        M53.constraint(f"Demand{j+1}", mf.Expr.add([x.index(i, j) for i in u_factories]), mf.Domain.equalsTo(demands[j]))

    # Constraint: capacity for each factory
    capacities = [6, 6]
    for i in u_factories:
        M53.constraint(f"Capacity{i+1}", mf.Expr.add([x.index(i, j) for j in v_factories]), mf.Domain.lessThan(capacities[i]))

    # Solving the model
    M53.solve()
    solution_status = M53.getPrimalSolutionStatus()
    minimal_cost = M53.primalObjValue()

    # Display solution
    print("Solution status:", solution_status)

    print("\nMaximal profit:", minimal_cost)

    print("\nOptimal quantities of engines to be transported from city U factory i to city V factory j:")
    for i in u_factories:
        for j in v_factories:
            print(f"x[{i+1},{j+1}] = {x.index(i,j).level()[0]}")

    print("\nExcedents in the factories from city U:")
    for i in u_factories:
        print(f"Excedent U Factory {i+1} = {capacities[i] - (x.index(i, 0).level()[0] + x.index(i, 1).level()[0] + x.index(i, 2).level()[0])}")

Solution status: SolutionStatus.Optimal

Maximal profit: 430.0

Optimal quantities of engines to be transported from city U factory i to city V factory j:
x[1,1] = 2.0
x[1,2] = 4.0
x[1,3] = 0.0
x[2,1] = 3.0
x[2,2] = 0.0
x[2,3] = 3.0

Excedents in the factories from city U:
Excedent U Factory 1 = 0.0
Excedent U Factory 2 = 0.0


In [ ]:
# Mixture Problem - List 5 Exercise 4

with mf.Model("list5_exercise4") as M54:
    # Decision variables s,c >= 0 and s,c are integers
    s = M54.variable("s", 1, mf.Domain.integral(mf.Domain.greaterThan(0.0)))
    c = M54.variable("c", 1, mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Ojective function: maximize profit
    M54.objective("MaximizeProfit", mf.ObjectiveSense.Maximize, 5*s + 4*c)

    # Constraint: supply of hours
    M54.constraint(f"HoursSupply", (1/6)*s + (1/5)*c, mf.Domain.lessThan(10))

    # Constraint: supply of leather
    M54.constraint(f"LeatherSupply", 2*s + c, mf.Domain.lessThan(78))

    # Solving the model
    M54.solve()
    solution_status = M54.getPrimalSolutionStatus()
    maximal_profit = M54.primalObjValue()
    optimal_s = int(round(s.level()[0]))
    optimal_c = int(round(c.level()[0]))

    # Display solution
    print("Solution status:", solution_status)

    print("\nMaximal profit:", maximal_profit)

    print("\nOptimal quantities of shoes and belts to be producted in one day by the facturer:")
    print(f"s = {optimal_s}")
    print(f"c = {optimal_c}")

    print("\nHours of work:")
    print(f"Quantity of hours spent on shoes = {(1/6)*optimal_s} hours")
    print(f"Quantity of hours spent on belts = {(1/5)*optimal_c} hours")

Solution status: SolutionStatus.Optimal

Minimal cost: 240.0

Optimal quantities of shoes and belts to be producted in one day by the facturer:
s = 24
c = 30

Hours of work:
Quantity of hours spent on shoes = 4.0 hours
Quantity of hours spent on belts = 6.0 hours


In [51]:
# Mixture Problem - List 5 Exercise 5

with mf.Model("list5_exercise5") as M55:
    #Decision Variables
    # qa, qb >= 0 and qa, qb are integers
    # qa = amount of galons of diesel A produced
    # qb = amount of galons of diesel B produced
    ga = M55.variable("ga", 1, mf.Domain.integral(mf.Domain.greaterThan(0.0)))
    gb = M55.variable("gb", 1, mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Ojective function: maximize profit
    M55.objective("MaximizeProfit", mf.ObjectiveSense.Maximize, 20*ga + 30*gb)

    # Constraint: supply of gasoline
    M55.constraint("Gasoline1Supply", 0.25*ga, mf.Domain.lessThan(500))
    M55.constraint("Gasoline2Supply", 0.25*ga + 0.5*gb, mf.Domain.lessThan(200))
    M55.constraint("Gasoline3Supply", 0.5*ga + 0.5*gb, mf.Domain.lessThan(200))

    # Solving the model
    M55.solve()
    solution_status = M55.getPrimalSolutionStatus()
    maximal_profit = M55.primalObjValue()
    optimal_ga = int(round(ga.level()[0]))
    optimal_gb = int(round(gb.level()[0]))

    # Display solution
    print("Solution status:", solution_status)
    
    print("\nMaximal profit:", maximal_profit)

    print("\nOptimal quantities of diesel A and diesel B to be produced:")
    print(f"ga = {optimal_ga}")
    print(f"gb = {optimal_gb}")

    print("\nAmount of gallons used for each type of gasoline:")
    print(f"Gasoline 1 = {0.25*optimal_ga} gallons")
    print(f"Gasoline 2 = {0.25*optimal_ga + 0.5*optimal_gb} gallons")
    print(f"Gasoline 3 = {0.5*optimal_ga + 0.5*optimal_gb} gallons")



Solution status: SolutionStatus.Optimal

Maximal profit: 12000.0

Optimal quantities of diesel A and diesel B to be produced:
ga = 0
gb = 400

Amount of gallons used for each type of gasoline:
Gasoline 1 = 0.0 gallons
Gasoline 2 = 200.0 gallons
Gasoline 3 = 200.0 gallons


In [ ]:
# Mixture Problem - List 5 Exercise 6

with mf.Model("list5_exercise6") as M56:
    #indexes
    diesel_types = [0, 1, 2]  # Diesel types
    oil_types = [0, 1, 2, 3]  # Oil types

    # Decision Variables
    x = M56.variable("x", [len(oil_types), len(diesel_types)], mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Objective function: maximize profit
    # diesel_selling_profit = M56.sum([
    #     x.slice([0, 0], [4, 1]).sum().mul(5.5),
    #     x.slice([0, 1], [4, 2]).sum().mul(4.5),
    #     x.slice([0, 2], [4, 3]).sum().mul(3.5)
    # ])

    # oil_buying_costs = M56.sum([
    #     x.slice([0, 0], [1, 3]).sum().mul(3),  # tipo 1
    #     x.slice([1, 0], [2, 3]).sum().mul(6),  # tipo 2
    #     x.slice([2, 0], [3, 3]).sum().mul(4),  # tipo 3
    #     x.slice([3, 0], [4, 3]).sum().mul(5)   # tipo 4
    # ])

    diesel_prices = [5.5, 4.5, 3.5]  # Prices of diesel types
    oil_prices = [3, 6, 4, 5]  # Prices of oil types

    revenue = sum(diesel_prices[j] * sum(x.index(i, j) for i in oil_types) for j in diesel_types)
    costs = sum(oil_prices[i] * sum(x.index(i, j) for j in diesel_types) for i in oil_types)

    M56.objective("MaximizeProfit", mf.ObjectiveSense.Maximize, revenue - costs)

    #M56.objective("MaximizeProfit", mf.ObjectiveSense.Maximize, diesel_selling_profit - oil_buying_costs)

    # Constraint: diesel composition
    # Diesel A
    M56.constraint("DemandA1", x.index(0,0), mf.Domain.lessThan(0.3*mf.Expr.add([x.index(i, 0) for i in oil_types])))
    M56.constraint("DemandA2", x.index(1,0)/mf.Expr.add([x.index(i, 0) for i in oil_types]), mf.Domain.greaterThan(0.4))
    M56.constraint("DemandA3", x.index(2,0)/mf.Expr.add([x.index(i, 0) for i in oil_types]), mf.Domain.lessThan(0.5))

    # Diesel B
    M56.constraint("DemandB1", x.index(0,1)/mf.Expr.add([x.index(i, 1) for i in oil_types]), mf.Domain.lessThan(0.5))
    M56.constraint("DemandB2", x.index(1,1)/mf.Expr.add([x.index(i, 1) for i in oil_types]), mf.Domain.greaterThan(0.1))

    # Diesel C
    M56.constraint("DemandC1", x.index(0,2)/mf.Expr.add([x.index(i, 2) for i in oil_types]), mf.Domain.lessThan(0.7))

    # Constraint: oil supply
    oil_supply = [3000, 2000, 4000, 1000]  # Supply of each oil type
    for i in oil_types:
        M56.constraint(f"OilSupply{i+1}", mf.Expr.add([x.index(i, j) for j in diesel_types]), mf.Domain.lessThan(oil_supply[i]))

    # Solving the model
    M56.solve()

    solution_status = M56.getPrimalSolutionStatus()
    maximal_profit = M56.primalObjValue()
    optimal_x = [[int(round(x.index(i,j).level()[0])) for j in diesel_types] for i in oil_types]

    # Display solution
    print("Solution status:", solution_status)
    print("\nMaximal profit:", maximal_profit)
    print("\nOptimal quantities of oil types to be used for each diesel type:")
    for i in oil_types:
        for j in diesel_types:
            print(f"x[{i+1},{j+1}] = {optimal_x[i][j]}")

TypeError: unsupported operand type(s) for /: 'float' and 'ExprWSum'